In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression

In [2]:
# the dataset is created using the random function by picking the appropriate ranges 
# The dataset is created using numpy and then converted into dataframe
  
# LMTD for counter-current flow calculating using the formula
#df['LMTD']= ((df['Thi'] - df['Tco']) - (df['Tho'] - df['Tci'])) / np.log((df['Thi'] - df['Tco']) / (df['Tho'] - df['Tci']) )

# to handle the anomaly of 0 denominator change Nan with approximate mean values
#35.2 was the mean value during first random iteration so we used that

#df.LMTD=df['LMTD'].fillna(35.2)

#df['Effectiveness'] = abs((df['Tco'] - df['Tci']) / (df['Thi'] - df['Tci']))


#A=2.12 # As area is constant it wont affect the efficiency thus we compressed the area to 10 times (Actual we considered is 21)

#df['Q_Ideal'] = df['UA']*A* df['LMTD']

# as we dont have the actual experimental values 
#we consider the bias including fouling factors plus environmental conditions and timely detoriation of the equipment
#This bias is based on our own intution after thorougly going through multiple research papers 
#l=[]
#for i in range(len(df)):
 #   l.append(random.uniform(0.55,0.92)* df.Q_Ideal[i])
#df['Q_Actual'] =l

#df['Efficiency'] = (df['Q_Actual'] / df['Q_Ideal'])*100

# Maintenance condition 
 #(1.Our hypothesis for the real-time scenario is either machine is old or has not followed scheduled maintenance properly
  #2.Due to hard water or some chemical reactions efficiency is reduced
  #3.Due to abrupt faliure in some component efficiency is dropped
  #4.Excessive fouling)

#l_foul=[]
#for i in range(len(df_foul)):
 #   l_foul.append(random.uniform(0.30,0.53)* df_foul.Q_Ideal[i])
#df_foul['Q_Maintenance'] =l_foul

#df_foul['Efficiency_Maintenance'] = df_foul['Q_Maintenance'] / df_foul['Q_Ideal']*100

#Accidental/Leakage Situation
#(1. This can cause due to input faliure
# 2.Mixing of the chemicals (like if water and oil mixes it can cause blast
 #3.Gas or fluid leak due to accidental failure(recent example: Vishakhapatnaam gas leak ) 
    
#    l_accidental=[]
#for i in range(len(df_accidental)):
#    l_accidental.append(random.uniform(0.1,0.30)* df_accidental.Q_Ideal[i])
#df_accidental['Q_Accidental'] =l_accidental

#df_accidental['Efficiency_Accidental'] = df_accidental['Q_Accidental'] / df_accidental['Q_Ideal']*100


In [3]:
df=pd.read_excel('HED_normal.xls')
df.head()

,Thi,Tci,Tho,Tco,UA,LMTD,Effectiveness,Q_Ideal,Q_Actual,Efficiency
0,111,18,70.767326,76.244375,0.194890,43.136556,0.626284,17.758747,15.599118,87.839069
1,88,18,61.764762,53.297435,0.171421,39.058606,0.504249,14.143527,12.863311,90.948392
2,90,8,67.385529,57.500574,0.160448,44.599946,0.603666,15.116298,10.306086,68.178637
3,122,28,74.697397,74.316927,0.184977,47.188520,0.492733,18.438738,10.707614,58.071292
4,118,30,56.295665,56.447936,0.177215,41.454741,0.300545,15.518627,12.103090,77.990724


In [4]:
X=df[['Q_Actual','Q_Ideal']]
y=df.Efficiency.values.reshape(-1,1)

In [5]:
from sklearn.svm import SVR
normal_svr=SVR(kernel='rbf')
normal_svr.fit(X,y)
normal_svr.score(X,y)*100

C:\Users\kedar\Anaconda3.7\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


93.9547830933897

In [6]:
#this function is rendered in our frontend model to demonstrate the efficiency
#the purpose behind using SVR is that it is best suited for small and non-linear data
normal_svr.predict([[109.072635,159.875254]])

array([76.17761518])

In [7]:
# Random Forest follows strict Decision boundries to get the better results we have purposefully used it 
rf=RandomForestRegressor()
X_N=df[['UA','LMTD','Efficiency']]
y_N=df[['Tho','Tco']]
# to output two values we have used multi-regressor
wrap = MultiOutputRegressor(rf)
wrap.fit(X_N,y_N)
wrap.score(X_N,y_N)*100
#df.head()

88.78523544344132

In [8]:
#This predicts the Temperature(out) for hot and cold 
#This predict function is called by an API
#This prediction will help you to confirm the asset performance based on efficiency
wrap.predict([[0.160,44.19,61.4]])

array([[59.32200756, 46.03064527]])

In [24]:
#df.head()

,Thi,Tci,Tho,Tco,UA,LMTD,Effectiveness,Q_Ideal,Q_Actual,Efficiency
0,111,18,70.767326,76.244375,0.194890,43.136556,0.626284,17.758747,15.599118,87.839069
1,88,18,61.764762,53.297435,0.171421,39.058606,0.504249,14.143527,12.863311,90.948392
2,90,8,67.385529,57.500574,0.160448,44.599946,0.603666,15.116298,10.306086,68.178637
3,122,28,74.697397,74.316927,0.184977,47.188520,0.492733,18.438738,10.707614,58.071292
4,118,30,56.295665,56.447936,0.177215,41.454741,0.300545,15.518627,12.103090,77.990724


 Maintenance condition 
 (1.Our hypothesis for the real-time scenario is either machine is old or has not followed scheduled maintenance properly
  2.Due to hard water or some chemical reactions efficiency is reduced
  3.Due to abrupt faliure in some component efficiency is dropped
  4.Excessive fouling)

In [10]:
df_foul = pd.read_excel('HED_maintenance .xls')

In [11]:
X=df_foul[['Q_Maintenance','Q_Ideal']]
y=df_foul.Efficiency_Maintenance.values.reshape(-1,1)

In [12]:
svr_foul=SVR(kernel='rbf')
svr_foul.fit(X,y)
svr_foul.score(X,y)*100

C:\Users\kedar\Anaconda3.7\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


92.29260251375457

In [13]:
svr_foul.predict([[172.533,374.116]])

array([42.61950543])

In [14]:
# Random Forest follows strict Decision boundries to get the better results we have purposefully used it 
rf=RandomForestRegressor()
X_M=df_foul[['UA','LMTD','Efficiency_Maintenance']]
y_M=df_foul[['Tho','Tco']]
wrap_M = MultiOutputRegressor(rf)
wrap_M.fit(X_M,y_M)
wrap_M.score(X_M,y_M)*100

88.62965259915069

In [15]:
#Temperature out prediction for hot and cold fluid
#This prediction will help you to confirm the asset performance based on efficiency
#This predict function is called by an API
wrap_M.predict([[0.185,47.19,36.797]])

array([[65.61490685, 51.27954514]])

In [16]:
#df_foul.head()

Accidental/Leakage Situation
(1. This can cause due to input faliure
 2.Mixing of the chemicals (like if water and oil mixes it can cause blast
 3.Gas or fluid leak due to accidental failure(recent example: Vishakhapatnaam gas leak ) 

In [17]:
df_accidental = pd.read_excel('HED_accidental.xls')
#df_accidental.head()

In [18]:
X=df_accidental[['Q_Accidental','Q_Ideal']]
y=df_accidental.Efficiency_Accidental.values.reshape(-1,1)

In [19]:
svr_accidental=SVR(kernel='rbf')
svr_accidental.fit(X,y)
svr_accidental.score(X,y)*100

C:\Users\kedar\Anaconda3.7\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


91.10186719765248

In [20]:
#df_accidental.head()

In [21]:
#Predicts accidental efficiency
svr_accidental.predict([[52.85,327.7]])

array([19.45452893])

In [22]:
rf=RandomForestRegressor()
X_A=df_accidental[['UA','LMTD','Efficiency_Accidental']]
y_A=df_accidental[['Tho','Tco']]
wrap_A = MultiOutputRegressor(rf)
wrap_A.fit(X_A,y_A)
wrap_A.score(X_A,y_A)*100

88.6981040346974

In [23]:
wrap_A.predict([[0.185,47.19,19.894]])

array([[60.48362296, 52.76018791]])